In [ ]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import tldextract
import unicodedata
from collections import Counter

# URL dataseto sudarymas

In [ ]:
data_benign_1 = pd.read_csv('dataset_B_05_2020.csv')
data_benign_1 = data_benign_1[data_benign_1['status'] == 'legitimate']
data_benign_1  = data_benign_1.sample(n=5000, random_state=1)
data_benign_1 = data_benign_1[['url']]

In [ ]:
data_benign_2 = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv')
data_benign_2 = data_benign_2[data_benign_2['label'] == 1]
data_benign_2  = data_benign_2.sample(n=5000, random_state=1)
data_benign_2 = data_benign_2[['url']]

In [ ]:
data_benign = pd.concat([data_benign_1, data_benign_2], ignore_index=True)
data_benign['phishing'] = 0

In [ ]:
data_phish_1 = pd.read_csv('verified_online.csv')
data_phish_1 = data_phish_1.head(10000)
data_phish_1  = data_phish_1.sample(n=5000, random_state=1)
data_phish_1 = data_phish_1[['url']]

In [ ]:
data_phish_2 = pd.read_csv('openphish.csv')
data_phish_2 = data_phish_2.head(10000)
data_phish_2  = data_phish_2.sample(n=5000, random_state=1)
data_phish_2 = data_phish_2[['url']]

In [ ]:
data_phish = pd.concat([data_phish_1, data_phish_2], ignore_index=True)
data_phish['phishing'] = 1

In [ ]:
full_data = pd.concat([data_benign, data_phish], ignore_index=True)

In [ ]:
full_data.to_csv("new_data.csv", index=False)

# Feature extraction

In [ ]:
data = pd.read_csv('drive/MyDrive/data/musu_dataset.csv')

In [ ]:
data = data[['url', 'phishing']]

In [ ]:
display(data.head(3))

,url,phishing
0,http://www.mathopenref.com/segment.html,0
1,http://www.onlinemathlearning.com/calculator.html,0
2,http://www.criaderodecaracoles.com/,0


In [ ]:
def feature_extraction(url):
  data_set = []

  # 1. url_len
  data_set.append(len(url))

  # 2. shortining_Service
  match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net', url)
  if match:
      data_set.append(1)
  else:
      data_set.append(0)

  # 3. cnt_at
  data_set.append(url.count('@'))

  # 4. double_slash_redirecting
  list = [x.start(0) for x in re.finditer('//', url)]
  if list[len(list)-1] > 6:
      data_set.append(1)
  else:
      data_set.append(0)

  # 5. prefix_suffix
  if re.findall(r"https?://[^\-]+-[^\-]+/", url):
      data_set.append(1)
  else:
      data_set.append(0)

  # 6.having_Sub_Domain
  if len(re.findall("\.", url)) == 1:
      data_set.append(0)
  else:
      data_set.append(1)

  # 7. HTTPS_token
  if re.findall(r"^https://", url):
      data_set.append(1)
  else:
      data_set.append(0)

  # 8. len_longest_word
  words =  re.findall(r'[a-zA-Z]+', url)
  max_word_length = max(map(len, words)) if words else 0
  data_set.append(max_word_length)

  # 9. cnt_num
  count_numbers = sum(c.isdigit() for c in url)
  data_set.append(count_numbers)

  # 10. cnt_dots (.)
  count_dots = url.count('.')
  data_set.append(count_dots)

  # 11. cnt_special_characters
  count_non_alpha = sum(not c.isalnum() and c not in ['.', '/'] for c in url)
  data_set.append(count_non_alpha)

  # 12. cnt_slashes
  count_slashes = url.count('/') - (2 if url.startswith("http") else 0)
  data_set.append(count_slashes)

  # 13. cnt_words
  letter_sequences = re.findall(r'[a-zA-Z]+', url)
  count_letter_sequences = len(letter_sequences)
  data_set.append(count_letter_sequences)

  # 14. has_port
  if len(re.findall(r":", url))>1:
      data_set.append(1)
  else:
      data_set.append(0)

  # 15. cnt_equals
  count_dots = url.count('=')
  data_set.append(count_dots)

  # 16. cnt_ampersand
  count_dots = url.count('&')
  data_set.append(count_dots)

  # 17. cnt_percent
  count_dots = url.count('%')
  data_set.append(count_dots)

  # 18. cnt_suffix
  count_dots = url.count('-')
  data_set.append(count_dots)

  # 19. TLD
  extracted = tldextract.extract(url)
  tld = extracted.suffix  # Gets 'co.uk'
  data_set.append(tld)

  # 20. cnt_qn
  count_dots = url.count('?')
  data_set.append(count_dots)

  # 21. cnt_plus
  count_dots = url.count('+')
  data_set.append(count_dots)

  # 22. cnt_underscore
  count_dots = url.count('_')
  data_set.append(count_dots)


  non_latin_count = 0
  latin = False
  non_latin = False

  #  Loop through each character in the URL
  for char in url:
    if char.isalpha():  # Only check letters
        name = unicodedata.name(char, "")
        if "LATIN" not in name:  # Non-Latin character
            non_latin_count += 1
            non_latin = True
        else:
            latin = True

  # Check if both Latin and non-Latin characters are present
  is_mixed = latin and non_latin

  # 23. cnt_non_latin
  data_set.append(non_latin_count)

  # 24. contains_mixed_characters
  data_set.append(1 if is_mixed else 0)

  # 25. cnt_num_replacements
  # Looks for cases where letters are replaced with numbers
  # Must be either that number surrounded by letters, or a letter on one side and special character on the other
  chars_to_count = ['0', '1', '7', '4', '3', '5', '8']
  pattern = r'(?<=[a-zA-Z])[0-4,7,3](?=[a-zA-Z!@#\$%\^&\*\(\)_\+\=\-])|(?<=[a-zA-Z!@#\$%\^&\*\(\)_\+\=\-])[0-4,7,3](?=[a-zA-Z])'
  count = sum(len(re.findall(f'(?<=[a-zA-Z]){char}(?=[a-zA-Z!@#\$%\^&\*\(\)_\+\=\-])|(?<=[a-zA-Z!@#\$%\^&\*\(\)_\+\=\-]){char}(?=[a-zA-Z])', url)) for char in chars_to_count)
  data_set.append(count)

  # 26. cnt_letter_replacements
  letters_to_count = ['l', 'O', 'e', 'A', 'c', 'I', 'S', 'B']
  pattern = r'(?<=[0-9])[lOeAcI](?=[0-9!@#\$%\^&\*\(\)_\+\=\-])|(?<=[0-9!@#\$%\^&\*\(\)_\+\=\-])[lOeAcI](?=[0-9])'
  let_count = sum(len(re.findall(f'(?<=[0-9])[{letter}](?=[0-9!@#\$%\^&\*\(\)_\+\=\-])|(?<=[0-9!@#\$%\^&\*\(\)_\+\=\-])[{letter}](?=[0-9])', url)) for letter in letters_to_count)
  data_set.append(let_count)

  return data_set

In [ ]:
col_names = ['url_len',
             'shortining_service',
             'cnt_at',
             'double_slash_redirecting',
             'preffix_suffix',
             'sub_domain',
             'https',
             'len_longest_word',
             'cnt_num',
             'cnt_dots',
             'cnt_special_char',
             'cnt_slash',
             'cnt_words',
             'port',
             'cnt_equals',
             'cnt_ampersand',
             'cnt_percent',
             'cnt_suffix',
             'TLD',
             'cnt_qn',
             'cnt_plus',
             'cnt_underscore',
             'cnt_non_latin',
             'contains_mixed_characters',
             'cnt_num_replacements',
             'cnt_letter_replacements']

data_features = data['url'].apply(feature_extraction).apply(pd.Series)
data_features.columns = col_names
data_set = pd.concat([data, data_features], axis=1)

In [ ]:
data_set.to_csv("drive/MyDrive/data/musu_dataset.csv", index=False)

In [ ]:
grouped_summary = data_set.drop(['url', 'TLD'],axis=1).groupby('phishing').mean()

display(grouped_summary)

,url_len,shortining_service,cnt_at,double_slash_redirecting,preffix_suffix,sub_domain,https,len_longest_word,cnt_num,cnt_dots,...,cnt_ampersand,cnt_percent,cnt_suffix,cnt_qn,cnt_plus,cnt_underscore,cnt_non_latin,contains_mixed_characters,cnt_num_replacements,cnt_letter_replacements
phishing,,,,,,,,,,,,,,,,,,,,,
0,37.3743,0.0645,0.0000,0.0002,0.0564,0.9567,0.7231,10.6056,0.7964,2.1848,...,0.0114,0.0405,0.6455,0.0174,0.0091,0.1436,0.0000,0.0000,0.0372,0.0133
1,43.3713,0.0342,0.0146,0.0004,0.3076,0.8265,0.7374,9.9381,3.9557,2.0543,...,0.0728,0.0354,1.1031,0.0372,0.0047,0.0686,0.0023,0.0003,0.5445,0.1202


In [ ]:
display(data_set.head(3))

,url,phishing,url_len,shortining_service,cnt_at,double_slash_redirecting,preffix_suffix,sub_domain,https,len_longest_word,...,cnt_percent,cnt_suffix,TLD,cnt_qn,cnt_plus,cnt_underscore,cnt_non_latin,contains_mixed_characters,cnt_num_replacements,cnt_letter_replacements
0,http://www.mathopenref.com/segment.html,0,39,0,0,0,0,1,0,11,...,0,0,com,0,0,0,0,0,0,0
1,http://www.onlinemathlearning.com/calculator.html,0,49,0,0,0,0,1,0,18,...,0,0,com,0,0,0,0,0,0,0
2,http://www.criaderodecaracoles.com/,0,35,0,0,0,0,1,0,19,...,0,0,com,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm , preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.11/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
data_set["TLD_encoded"] = label_encoder.fit_transform(data_set["TLD"])
data_set = data_set.drop(columns=["TLD"])

In [ ]:
X = data_set.drop(columns=['url', 'phishing'])
Y = data_set['phishing']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=20, random_state=42)
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9417


In [ ]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_classifier.feature_importances_
})

# Sort features by importance (Descending order)
feature_importances = feature_importances.sort_values(by="Importance", ascending=False)

# Display top important features
print(feature_importances.head(10))

             Feature  Importance
11         cnt_slash    0.203209
25       TLD_encoded    0.163155
0            url_len    0.125716
7   len_longest_word    0.087241
12         cnt_words    0.076064
8            cnt_num    0.069561
17        cnt_suffix    0.065974
10  cnt_special_char    0.040485
4     preffix_suffix    0.035157
6              https    0.028944


In [ ]:
data_test = pd.read_csv('drive/MyDrive/data/dataset_small_2020.csv')

In [ ]:
data_test.head(1)

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,qty_at_url,qty_and_url,qty_exclamation_url,qty_space_url,...,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_google_index,domain_google_index,url_shortened,phishing
0,2,0,0,0,0,0,0,0,0,0,...,1,4,2,3598,0,0,0,0,0,0


In [ ]:
X = data_test.drop(columns=['phishing'])
Y = data_test['phishing']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=20, random_state=42)

# Train the model
rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9558


In [ ]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_classifier.feature_importances_
})

# Sort features by importance (Descending order)
feature_importances = feature_importances.sort_values(by="Importance", ascending=False)

# Display top important features
print(feature_importances.head(10))

                    Feature  Importance
57         directory_length    0.135183
100  time_domain_activation    0.083995
18               length_url    0.047481
3             qty_slash_url    0.041400
55     qty_dollar_directory    0.041099
43      qty_slash_directory    0.040602
71        qty_asterisk_file    0.038646
105            ttl_hostname    0.037391
75              file_length    0.034848
101  time_domain_expiration    0.032029
